In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import talib

Populating the interactive namespace from numpy and matplotlib


In [2]:
def crop(df, start='2016-03-01 10:00:00', end='2016-09-01 10:00:00'):
    df = df[df[df.dt == start].index[0]:df[df.dt == end].index[0]]
    df.index = pd.RangeIndex(df.shape[0])
    return df

In [3]:
PATH_TO_DATA = os.getcwd()
PATH_TO_DATA = PATH_TO_DATA[:-PATH_TO_DATA[::-1].find("/")]
PATH_TO_DATA += "data/shares/"
df_emr = crop(pd.read_csv(PATH_TO_DATA + "EMR.csv", sep=','))
df_jbl = crop(pd.read_csv(PATH_TO_DATA + "JBL.csv", sep=','))
df_met = crop(pd.read_csv(PATH_TO_DATA + "MET.csv", sep=','))
df_see = crop(pd.read_csv(PATH_TO_DATA + "SEE.csv", sep=','))

PATH_TO_DATA += "semiconductors/"
df_mu = crop(pd.read_csv(PATH_TO_DATA + "MU.csv", sep=','))
df_amat = crop(pd.read_csv(PATH_TO_DATA + "AMAT.csv", sep=','))
df_amd = crop(pd.read_csv(PATH_TO_DATA + "AMD.csv", sep=','))
df_intc = crop(pd.read_csv(PATH_TO_DATA + "INTC.csv", sep=','))
df_klac = crop(pd.read_csv(PATH_TO_DATA + "KLAC.csv", sep=','))
df_lrcx = crop(pd.read_csv(PATH_TO_DATA + "LRCX.csv", sep=','))
df_mchp = crop(pd.read_csv(PATH_TO_DATA + "MCHP.csv", sep=','))
df_nvda = crop(pd.read_csv(PATH_TO_DATA + "NVDA.csv", sep=','))
df_ter = crop(pd.read_csv(PATH_TO_DATA + "TER.csv", sep=','))

In [4]:
dfs = [df_emr, df_jbl, df_met, df_see, df_mu, 
       df_amat, df_amd, df_intc, df_klac, 
       df_lrcx, df_mchp, df_nvda, df_ter]

prices = []
for df in dfs:
    prices.append(df['open'].values)

In [5]:
class IndicatorMarks():
    
    
    def __init__(self):
        pass
        
        
    def macd_marks(self, MACD):
        periods = len(MACD)
        temp = ['hold'] * periods
        for i in range(1, periods):
            if (MACD[i] > 0 and MACD[i-1] < 0):
                temp[i] = 'buy'
            elif (MACD[i] < 0 and MACD[i-1] > 0):
                temp[i] = 'sell'
        return temp
    
    
    def rsi_marks(self, rsi, buy_threshold=30, sell_threshold=70):
        periods = len(rsi)
        temp = ['hold'] * periods
        for i in range(1, periods):
            if (rsi[i] < buy_threshold):
                temp[i] = 'buy'
            elif (rsi[i] > sell_threshold):
                temp[i] = 'sell'
        return temp

In [6]:
class SingleInstrumentBacktest:
    
    df = None #dataframe
    price = None #open or close
    periods = None #number of periods
    balance = None #balance on current period of time
    fee = None #taker/maker fee
    stoploss = None
    takeprofit = None
    hold = 0 #current balance of an instrument
    i = 0 #current period index
    balance_on_buy = None #balance when bought to know current loss/profit
    wait = 0 #no action for "10" (e.g.) periods after stoploss/takeprofit
    
    
    def __init__(self, df, initial_balance=100, fee=0.1, stoploss=0.03, takeprofit=0.05):
        self.df = df.copy()
        self.price = self.df['open'].values
        self.periods = df.shape[0]
        self.df.index = pd.RangeIndex(self.periods) #refresh index
        self.balance = np.zeros(self.periods)
        self.balance[0] = initial_balance
        self.fee = fee
        self.stoploss = stoploss
        self.takeprofit = takeprofit
        self.add_indicator_marks()
        
        
    def add_indicator_marks(self):
        im = IndicatorMarks()
        self.df['macd'] = talib.EMA(self.price, timeperiod=13) - talib.EMA(self.price, timeperiod=26)
        self.df['macd_mark'] = im.macd_marks(self.df['macd'].values)
        self.df['rsi_14'] = talib.RSI(self.price, timeperiod=14)
        self.df['rsi_14_mark'] = im.rsi_marks(self.df['rsi_14'].values, 30, 70)
    
    
    def buy_condition(self):
        #if (self.df['rsi_14_mark'][self.i] == 'buy' and self.wait == 0):
        if (self.df['custom_signal'][self.i] == 'buy' and self.wait == 0):
            return True
        if (self.wait > 0):
            self.wait -= 1
        return False
    
    
    def sell_condition(self):
        #if (self.df['rsi_14_mark'][self.i] == 'sell'):
        if (self.df['custom_signal'][self.i] == 'sell'):
            return True
        #stoploss/takeprofit
        current_trade_loss = 1.0 - self.balance[self.i] / self.balance_on_buy
        if (current_trade_loss >= self.stoploss or -1.0 * current_trade_loss >= self.takeprofit):
            self.wait = 8
            return True
        return False
    
    
    def step(self):
        if self.hold == 0:
            if (self.buy_condition() == True):
                self.hold = self.balance[self.i-1] / self.price[self.i] * (1.0 - self.fee/100.0)
                self.balance_on_buy = self.balance[self.i-1]
            self.balance[self.i] = self.balance[self.i-1]
        else:
            self.balance[self.i] = self.hold * self.price[self.i] * (1.0 - self.fee/100.0)
            if (self.sell_condition() == True):
                self.hold = 0
        
        
    def full_backtest(self, print_metrics=True, show_balance=True):
        for _ in range(1, self.periods):
            self.i += 1
            self.step()
        if print_metrics == True:
            print(self.get_metrics())
        if show_balance == True:
            self.show_balance()
        return self.balance
    
    
    def show_balance(self):
        plt.plot(range(self.periods), self.balance)
        
        
    def get_metrics(self, digits=3):
        risk_free_return = 0
        self.df['return'] = self.balance / self.balance[0]
        max_drawdown = 1.0 - min(self.df['return'].values)
        self.df['return'] = self.df['return'].pct_change(1)
        sharpe = (self.df['return'].mean() - risk_free_return) / self.df['return'].std()
        sharpe *= self.periods ** 0.5 #annualize
        #sharpe = np.mean(ret) / np.std(ret)
        return {"profit factor" : round(self.balance[self.periods-1]/self.balance[0], digits), 
                "sharpe ratio" : round(sharpe, digits),
                "max drawdown" : str(round(max_drawdown * 100, digits-2)) + "%"}

In [7]:
class BasketTrading():
    
    prices = None
    factors = []
    ranks = None
    rebalance = 20
    
    
    def __init__(self, prices, rebalance=20):
        self.prices = prices
        self.form_factors()
        self.rebalance = rebalance
    
    
    def factor(self, p):
        return talib.RSI(p, timeperiod=14)
    
    
    def form_factors(self):
        for p in prices:
            self.factors.append(self.factor(p))
            
            
    def form_ranks(self):
        n = len(prices)
        m = len(prices[0])
        self.ranks = [['hold'] * m] * n
        self.ranks = np.array(self.ranks)
        for i in range(14, m):
            if (i % self.rebalance == 0):
                temp = []
                for j in range(n):
                    temp.append(self.factors[j][i])
                temp_df = pd.DataFrame(index=range(n), columns=['factors'])
                temp_df['factors'] = temp
                temp_df = temp_df.sort_values('factors')
            
                for ind in temp_df.index[:int(n/2)+1]:
                    self.ranks[ind][i] = 'buy'
                for ind in temp_df.index[int(n/2)+1:]:
                    self.ranks[ind][i] = 'sell'
        return self.ranks
    
    
    def trade(self):
        balances = []
        for i in range(len(prices)):
            df = pd.DataFrame(index = range(len(prices[i])), columns=['open', 'custom_signal'])
            df['open'] = self.prices[i]
            df['custom_signal'] = self.ranks[i]
            sib = SingleInstrumentBacktest(df, stoploss=1000, takeprofit=1000)
            balances.append((sib.full_backtest(show_balance=False), sib.get_metrics()))
        return balances


In [8]:
bt = BasketTrading(prices)
ranks = bt.form_ranks()

In [9]:
balances = bt.trade()

{'profit factor': 1.089, 'sharpe ratio': 0.799, 'max drawdown': '1.9%'}
{'profit factor': 1.02, 'sharpe ratio': 0.211, 'max drawdown': '13.9%'}
{'profit factor': 1.274, 'sharpe ratio': 1.634, 'max drawdown': '0.2%'}
{'profit factor': 0.996, 'sharpe ratio': 0.053, 'max drawdown': '11.1%'}
{'profit factor': 1.18, 'sharpe ratio': 0.798, 'max drawdown': '15.2%'}
{'profit factor': 1.008, 'sharpe ratio': 0.123, 'max drawdown': '7.9%'}
{'profit factor': 1.573, 'sharpe ratio': 1.365, 'max drawdown': '5.4%'}
{'profit factor': 1.111, 'sharpe ratio': 1.102, 'max drawdown': '2.3%'}
{'profit factor': 1.02, 'sharpe ratio': 0.277, 'max drawdown': '1.9%'}
{'profit factor': 1.08, 'sharpe ratio': 0.689, 'max drawdown': '4.7%'}
{'profit factor': 1.31, 'sharpe ratio': 2.302, 'max drawdown': '0.2%'}
{'profit factor': 1.178, 'sharpe ratio': 1.267, 'max drawdown': '2.6%'}
{'profit factor': 0.976, 'sharpe ratio': -0.151, 'max drawdown': '8.4%'}


In [10]:
#6 months
avg_profit_factor = 0
for x in balances:
    avg_profit_factor += x[1]['profit factor']
print(avg_profit_factor / len(balances)) 
#seems to be good but benchmark "buy and hold" gives 1.42 so it's not

1.1396153846153845
